In [108]:
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy

In [109]:
print("pandas version:", pd.__version__)
print("sqlalchemy version:", sqlalchemy.__version__)

pandas version: 2.2.2
sqlalchemy version: 1.4.53


In [110]:
username = "admin"
password = '1234'
host = 'localhost'
port = 5433
database = 'pmudb'

In [111]:
conn = psycopg2.connect(
    dbname=database,
    user=username,
    password=password,
    host=host,
    port=port
)

In [223]:
# NERC 기준 설정
from datetime import timedelta

frequency_lower_threshold = -0.45  # 주파수 하락 이벤트 기준 (-0.45 Hz 이하)
frequency_upper_threshold = 1.0    # 주파수 상승 이벤트 기준 (+1.0 Hz 이상)
rocof_threshold = 0.124            # 주파수 변화율 이벤트 기준
base_voltage_threshold = 0.15           # 전압 이벤트 기준 (85% 이하)
base_voltage = 66
duration_threshold = timedelta(seconds=5)  # 전압 이벤트 지속 시간
tkeo_threshold = 0.05               # TKEO 임계값
fetch_interval = timedelta(seconds=1)  # 데이터 수집 주기

In [136]:
def select_from_btw_time(connection, table_name, start_time, end_time):
    query = f""" 
            SELECT CAST(value as numeric), key, CAST(timestamp as timestamp) FROM {table_name} 
            WHERE timestamp BETWEEN '{start_time}' AND '{end_time}'
            """
    df = pd.read_sql(query, con=connection)
    return df.pivot(index="timestamp", columns='key', values='value')

In [137]:
def select_from_btw_time_with_key(connection, table_name, keys, start_time, end_time):
    query = f"""
        SELECT timestamp, key, CAST(value AS numeric) AS value
        FROM {table_name}
        WHERE timestamp BETWEEN '{start_time}' AND '{end_time}'
        AND key IN {keys}
    """

    df = pd.read_sql(query, con=connection)
    return df.pivot(index="timestamp", columns='key', values='value')

In [145]:
from datetime import datetime, timedelta
end_time = datetime.now() - timedelta(hours=9)
start_time = end_time - timedelta(seconds=1)
print(start_time, end_time)
df = select_from_btw_time(connection=conn, table_name="pmu_data", start_time=start_time, end_time=end_time)
df.head()

2024-10-29 17:35:29.360895 2024-10-29 17:35:30.360895


/var/folders/l0/gz2yp52d3256hc7bdtsgkyvh0000gn/T/ipykernel_26745/3792522060.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


key,Current_I1,Current_I1_Angle,Current_I2,Current_I2_Angle,Current_I3,Current_I3_Angle,Current_I4,Current_I4_Angle,Current_I5,Current_I5_Angle,...,DeFrequency,Frequency,Voltage_R,Voltage_R_Angle,Voltage_S,Voltage_S_Angle,Voltage_T,Voltage_T_Angle,Voltage_Z,Voltage_Z_Angle
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-10-29 17:35:29.371,0.670395,178.609966,0.669268,178.579954,0.669792,178.599953,0.670502,178.619952,0.667247,178.599953,...,-0.000999,60.015003,66.148888,1.260000,66.268410,1.280000,66.298141,1.300000,66.114006,1.280000
2024-10-29 17:35:29.408,0.668394,-93.839976,0.667267,-93.889980,0.668793,-93.869981,0.668501,-93.869974,0.665249,-93.899979,...,-0.003998,59.984001,65.988991,88.679980,66.108315,88.699978,66.137955,88.719984,65.954201,88.699978
2024-10-29 17:35:29.440,0.673397,9.769998,0.672269,9.699998,0.673791,9.699998,0.673505,9.729997,0.670243,9.719999,...,-0.001999,59.987003,66.528656,-167.599960,66.648628,-167.579962,66.678589,-167.559963,66.493576,-167.579962
2024-10-29 17:35:29.477,0.672396,71.809979,0.671268,71.769982,0.671792,71.769982,0.672504,71.819986,0.669244,71.789981,...,-0.006001,59.968002,66.438713,-105.629975,66.568581,-105.609976,66.588486,-105.589978,66.403679,-105.609976
2024-10-29 17:35:29.507,0.666393,-159.139965,0.666266,-159.199962,0.666793,-159.169963,0.666499,-159.139965,0.663251,-159.189976,...,0.004002,60.026001,65.819099,23.419995,65.948219,23.439993,65.967751,23.459994,65.784393,23.439993


In [144]:
df = select_from_btw_time_with_key(connection=conn, table_name="pmu_data", keys="('Voltage_R', 'Voltage_S', 'Voltage_T')", start_time=start_time, end_time=end_time)
df.head()

/var/folders/l0/gz2yp52d3256hc7bdtsgkyvh0000gn/T/ipykernel_26745/3058080543.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


key,Voltage_R,Voltage_S,Voltage_T
timestamp,,,
2024-10-29 17:35:23.188,65.998985,66.118324,66.137955
2024-10-29 17:35:23.224,65.779121,65.898193,65.927704
2024-10-29 17:35:23.261,66.018974,66.148338,66.167984
2024-10-29 17:35:23.294,66.478683,66.608604,66.628525
2024-10-29 17:35:23.325,66.398735,66.518555,66.538422


In [159]:
def detect_frequency_event(df, frequency_column, base_frequency, frequency_lower_threshold, frequency_upper_threshold):
    events = []
    if not df.empty:
        freq_values = df[frequency_column].values
        freq_deviation = freq_values - base_frequency  # 기준 주파수 60 Hz에서의 편차 계산

        # 주파수 하락 이벤트 (-0.45 Hz 이하)
        for i, deviation in enumerate(freq_deviation):
            if deviation < frequency_lower_threshold:
                event_time = df.index[i]
                events.append({
                    "timestamp": event_time,
                    "event_type": "Frequency Drop Event",
                    "value": freq_values[i],
                    "description": f"Frequency dropped below {60 + frequency_lower_threshold} Hz"
                })

        # 주파수 상승 이벤트 (+1.0 Hz 이상)
        for i, deviation in enumerate(freq_deviation):
            if deviation > frequency_upper_threshold:
                event_time = df.index[i]
                events.append({
                    "timestamp": event_time,
                    "event_type": "Frequency Rise Event",
                    "value": freq_values[i],
                    "description": f"Frequency exceeded {60 + frequency_upper_threshold} Hz"
                })
    return events


In [160]:
fr_df = detect_frequency_event(df, base_frequency=60, frequency_column="Frequency", frequency_lower_threshold=-0.1, frequency_upper_threshold=0.05)
fr_df

[{'timestamp': Timestamp('2024-10-29 17:35:29.723000'),
  'event_type': 'Frequency Rise Event',
  'value': 60.07100296020508,
  'description': 'Frequency exceeded 60.05 Hz'},
 {'timestamp': Timestamp('2024-10-29 17:35:29.755000'),
  'event_type': 'Frequency Rise Event',
  'value': 60.05700302124024,
  'description': 'Frequency exceeded 60.05 Hz'},
 {'timestamp': Timestamp('2024-10-29 17:35:30.143000'),
  'event_type': 'Frequency Rise Event',
  'value': 60.073001861572266,
  'description': 'Frequency exceeded 60.05 Hz'}]

In [165]:
def detect_rocof_event(df, rocof_column:str, rocof_threshold):
    events = []
    if not df.empty:
        rocof_values = df[rocof_column].values
        for i, deviation in enumerate(rocof_values):
            if abs(deviation) > rocof_threshold:
                event_time = df.index[i]
                events.append({
                    "timestamp": event_time,
                    "event_type": "ROCOF Event",
                    "value": rocof_values[i],
                    "description": f"Rate of Frequency is changed over {rocof_threshold} Hz"
                })
    return events


In [170]:
rocof_df = detect_rocof_event(df=df, rocof_column="DeFrequency", rocof_threshold=0.002)
rocof_df

[{'timestamp': Timestamp('2024-10-29 17:35:29.408000'),
  'event_type': 'ROCOF Event',
  'value': -0.003997802734375,
  'description': 'Rate of Frequency is changed over 0.002 Hz'},
 {'timestamp': Timestamp('2024-10-29 17:35:29.477000'),
  'event_type': 'ROCOF Event',
  'value': -0.0060005187988281,
  'description': 'Rate of Frequency is changed over 0.002 Hz'},
 {'timestamp': Timestamp('2024-10-29 17:35:29.507000'),
  'event_type': 'ROCOF Event',
  'value': 0.0040016174316406,
  'description': 'Rate of Frequency is changed over 0.002 Hz'},
 {'timestamp': Timestamp('2024-10-29 17:35:29.538000'),
  'event_type': 'ROCOF Event',
  'value': 0.0030021667480468,
  'description': 'Rate of Frequency is changed over 0.002 Hz'},
 {'timestamp': Timestamp('2024-10-29 17:35:29.649000'),
  'event_type': 'ROCOF Event',
  'value': 0.0069999694824218,
  'description': 'Rate of Frequency is changed over 0.002 Hz'},
 {'timestamp': Timestamp('2024-10-29 17:35:29.723000'),
  'event_type': 'ROCOF Event',
  

In [212]:
df = select_from_btw_time_with_key(connection=conn, table_name="pmu_data", keys="('Voltage_R', 'Voltage_S', 'Voltage_T')", start_time=start_time-timedelta(seconds=5), end_time=end_time)

/var/folders/l0/gz2yp52d3256hc7bdtsgkyvh0000gn/T/ipykernel_26745/3058080543.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


In [225]:
voltage_threshold_percent = 0.1       # 전압 변화 기준 ±10%
base_voltage

66

In [232]:
# 전압 변화 감지 함수
def detect_voltage_event(df, voltage_phase, base_voltage, voltage_threshold_percent):
    events = []
    voltage_data = df[voltage_phase]
    
    if not voltage_data.empty:
        # 전압 변화가 ±10% 초과하는 경우 감지
        voltage_deviation = abs(voltage_data - base_voltage)
        deviation_threshold = base_voltage * voltage_threshold_percent
        
        for i, deviation in enumerate(voltage_deviation):
            if deviation > deviation_threshold:
                event_time = voltage_data.index[i]
                events.append({
                    "timestamp": event_time,
                    "event_type": f"Voltage Event {voltage_phase}",
                    "value": voltage_data.iloc[i],
                    "description": f"Voltage deviation exceeded ±10% on {voltage_phase}"
                })
    return events

In [249]:
voltage_ano = detect_voltage_event(df, voltage_phase="Voltage_R", base_voltage=base_voltage, voltage_threshold_percent=0.009)

In [97]:
delta_f = df[df.key == "Frequency"].value - base_frequency

In [99]:
def detect_tkeo_event(df, frequency_column, base_frequecy, tkeo_threshold):
    

In [102]:
# SELECT CAST(value AS numeric), CAST(timestamp AS timestamp)
# FROM pmu_data 
# WHERE key = 'Frequency'
# ORDER BY CAST(timestamp AS timestamp) desc
# limit 100;